# Principal Component Analysis (PCA)

In [ ]:
import numpy as np
import pandas as pd
import time

### Create synthetic dataset

Here we create a rank 1 matrix.  All rows are a multiple of a single vector.  PCA should identify this vector direction as capturing all the variance in the data.

In [ ]:
num_vecs = 100000
dim = 2000
dtype = 'float32'
vec = np.random.randn(dim).reshape([1,dim])
arr = np.random.randn(num_vecs).reshape([num_vecs,1])
data = arr * vec
data = data.astype(dtype)

### Convert dataset to Spark DataFrame

In [ ]:
pd_data = pd.DataFrame({"features": list(data)})
df = spark.createDataFrame(pd_data)

### We will use this function to build both the Spark RAPIDS ML (GPU) and Spark ML (CPU) linear estimator objects, demonstrating the common API

In [ ]:
def build_pca_estimator(estimator_class):
    return ( 
            estimator_class()
            .setK(2)
            .setInputCol("features")
           )

## Spark RAPIDS ML (GPU)

In [ ]:
from spark_rapids_ml.feature import PCA
gpu_pca = build_pca_estimator(PCA)

PCA estimator can be persisted and reloaded.

In [ ]:
estimator_path = "/tmp/pca-estimator"

In [ ]:
gpu_pca.write().overwrite().save(estimator_path)
gpu_pca_loaded = PCA.load(estimator_path)

### Fit

In [ ]:
start_time = time.time()
gpu_model = gpu_pca_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

Verify that all variance is captured by the first component.

In [ ]:
gpu_model.explainedVariance

In [ ]:
gpu_model.mean[0:10]

In [ ]:
gpu_model.pc

In [ ]:
gpu_model.pc.toArray()[0:10,:]

Verify that the computed largest principal component vector is a multiple of the original vector used to compute the rows of the rank 1 data.

In [ ]:
np.divide(vec,gpu_model.pc.toArray()[:,0])[0:10]

### Transform

In [ ]:
model_path = "/tmp/pca-model"

In [ ]:
gpu_model.write().overwrite().save(model_path)

In [ ]:
gpu_model_loaded = gpu_model.load(model_path)

In [ ]:
gpu_model_loaded.mean[0:10]

In [ ]:
gpu_model_loaded.explainedVariance

In [ ]:
gpu_model_loaded.pc.toArray()[0:10,:]

In [ ]:
transformed_df = gpu_model.setOutputCol("transformed").transform(df.repartition(gpu_model.num_workers))

In [ ]:
transformed_df.printSchema()

In [ ]:
transformed_df.count()

In [ ]:
transformed_df.show(10)

## Spark ML (CPU)

In [ ]:
from pyspark.ml.feature import PCA
cpu_pca = build_pca_estimator(PCA)

Convert array sql type to VectorUDT expected by Spark ML algos (Note: Spark RAPIDS ML also accepts VectorUDT Dataframes in addition to array type Dataframe above, along with a scalar column format - see docs).

In [ ]:
from pyspark.ml.functions import array_to_vector

In [ ]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

In [ ]:
vector_df.printSchema()

### Fit

In [ ]:
start_time = time.time()
cpu_pca_model = cpu_pca.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

In [ ]:
cpu_pca_model.explainedVariance

In [ ]:
pc = cpu_pca_model.pc

In [ ]:
np.divide(vec,pc.toArray()[:,0])[0][0:10]